In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import sys
import numpy as np
import pandas as pd
#from scipy.misc import imread
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import time

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

from sklearn.utils import shuffle

import numpy.random as rng

Using TensorFlow backend.


In [3]:
train_path = '/content/drive/My Drive/SOC_OneShot/train'
val_path = '/content/drive/My Drive/SOC_OneShot/val'
model_path = '/content/drive/My Drive/SOC_OneShot/weights/'

In [7]:
def loadimg(path):
  X = []
  y = []
  curr_y = 0
  classes = {}
  for letter in os.listdir(path):
    category_images=[]
    letter_path = os.path.join(path, letter)
                
    for filename in os.listdir(letter_path):
      image_path = os.path.join(letter_path, filename)
      image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
      category_images.append(image)
      y.append(curr_y)
    try:
      X.append(np.stack(category_images))
    except ValueError as e:
      print(e)
      print("error - category_images:", category_images)
    curr_y += 1
    classes[letter] = curr_y - 1
  y = np.vstack(y)
  X = np.stack(X)
  return X,y,classes

In [8]:
X, y, classes = loadimg(train_path)

In [9]:
X.shape

(15, 2, 200, 200)

In [10]:
y.shape

(30, 1)

In [11]:
classes.keys()

dict_keys(['I', 'R', 'O', 'SPACE', 'Y', 'DEL', 'K', 'L', 'N', 'W', 'B', 'F', 'D', 'A', 'C'])

In [12]:
Xval, yval, classesval = loadimg(val_path)

In [18]:
def initialize_weights(shape, name=None, dtype = float):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

In [22]:
def initialize_bias(shape, name=None, dtype = float):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [43]:
def get_siamese_model(input_shape):
    """
        Model architecture
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), activation='relu',
                     kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [44]:
model = get_siamese_model((200, 200, 1))
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
sequential_5 (Sequential)       (None, 4096)         27675584    input_9[0][0]                    
                                                                 input_10[0][0]                   
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 4096)         0           sequential_5[1][0]         

In [45]:
optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy",optimizer=optimizer)

In [37]:
def get_batch(X,classes,batch_size,s="train"):
    """
    Create batch of n pairs, half same class, half different class
    """
    if s == 'train':
        X = X
        categories = classes
    else:
        X = Xval
        categories = classesval
    n_classes, n_examples, w, h = X.shape
    
    # randomly sample several classes to use in the batch
    categories = rng.choice(n_classes,size=(batch_size,),replace=False)
    
    # initialize 2 empty arrays for the input image batch
    pairs=[np.zeros((batch_size, h, w,1)) for i in range(2)]
    
    # initialize vector for the targets
    targets=np.zeros((batch_size,))
    
    # make one half of it '1's, so 2nd half of batch has same class
    targets[batch_size//2:] = 1
    for i in range(batch_size):
        category = categories[i]
        idx_1 = rng.randint(0, n_examples)
        pairs[0][i,:,:,:] = X[category, idx_1].reshape(w, h, 1)
        idx_2 = rng.randint(0, n_examples)
        
        # pick images of same class for 1st half, different for 2nd
        if i >= batch_size // 2:
            category_2 = category  
        else: 
            # add a random number to the category modulo n classes to ensure 2nd image has a different category
            category_2 = (category + rng.randint(1,n_classes)) % n_classes
        
        pairs[1][i,:,:,:] = X[category_2,idx_2].reshape(w, h,1)
    
    return pairs, targets

In [27]:
def generate(batch_size, s="train"):
    """
    a generator for batches, so model.fit_generator can be used.
    """
    while True:
        pairs, targets = get_batch(batch_size,s)
        yield (pairs, targets)


In [39]:
def make_oneshot_task(N, s="val", language=None):
    """Create pairs of test image, support set for testing N way one-shot learning. """
    Xval, yval, classesval = loadimg(val_path)
    if s == 'train':
        X = X
        categories = classes
    else:
        X = Xval
        categories = classesval
    n_classes, n_examples, w, h = X.shape
    
    indices = rng.randint(0, n_examples,size=(N,))
    if language is not None: # if language is specified, select characters for that language
        low, high = categories[language]
        if N > high - low:
            raise ValueError("This language ({}) has less than {} letters".format(language, N))
        categories = rng.choice(range(low,high),size=(N,),replace=False)

    else: # if no language specified just pick a bunch of random letters
        categories = rng.choice(range(n_classes),size=(N,),replace=False)           
    
    true_category = categories[0]
    ex1, ex2 = rng.choice(n_examples,replace=False,size=(2,))
    test_image = np.asarray([X[true_category,ex1,:,:]]*N).reshape(N, w, h,1)
    support_set = X[categories,indices,:,:]
    support_set[0,:,:] = X[true_category,ex2]
    support_set = support_set.reshape(N, w, h,1)
    targets = np.zeros((N,))
    targets[0] = 1
    targets, test_image, support_set = shuffle(targets, test_image, support_set)
    pairs = [test_image,support_set]
    return pairs, targets


In [30]:
def test_oneshot(model, N, k, s = "val", verbose = 0):
  """Test average N way oneshot learning accuracy of a siamese neural net over k one-shot tasks"""
  n_correct = 0
  if verbose:
      print("Evaluating model on {} random {} way one-shot learning tasks ... \n".format(k,N))
  for i in range(k):
      inputs, targets = make_oneshot_task(N,s)
      probs = model.predict(inputs)
      if np.argmax(probs) == np.argmax(targets):
          n_correct+=1
  percent_correct = (100.0 * n_correct / k)
  if verbose:
      print("Got an average of {}% {} way one-shot learning accuracy \n".format(percent_correct,N))
  return percent_correct

In [47]:
# Hyper parameters
evaluate_every = 100 # interval for evaluating on one-shot tasks
batch_size = 2
n_iter = 5000 # No. of training iterations
N_way = 9 # how many classes for testing one-shot tasks
n_val = 100 # how many one-shot tasks to validate on
best = -1

In [48]:
t_start = time.time()
for i in range(1, n_iter+1):
  (inputs,targets) = get_batch(X,classes,batch_size)
  loss = model.train_on_batch(inputs, targets)
  if i % evaluate_every == 0:
    print("\n ------------- \n")
    print("Time for {0} iterations: {1} mins".format(i, (time.time()-t_start)/60.0))
    print("Train Loss: {0}".format(loss)) 
    val_acc = test_oneshot(model, N_way, n_val, verbose=True)
    model.save_weights(os.path.join(model_path, 'weights.{}.h5'.format(i)))
    if val_acc >= best:
      print("Current best: {0}, previous best: {1}".format(val_acc, best))
      best = val_acc


 ------------- 

Time for 100 iterations: 0.13234188159306845 mins
Train Loss: 0.37487107515335083
Evaluating model on 100 random 9 way one-shot learning tasks ... 

Got an average of 93.0% 9 way one-shot learning accuracy 

Current best: 93.0, previous best: -1

 ------------- 

Time for 200 iterations: 0.48914155960083006 mins
Train Loss: 0.617706298828125
Evaluating model on 100 random 9 way one-shot learning tasks ... 

Got an average of 89.0% 9 way one-shot learning accuracy 


 ------------- 

Time for 300 iterations: 0.8443007628122966 mins
Train Loss: 0.593525767326355
Evaluating model on 100 random 9 way one-shot learning tasks ... 

Got an average of 90.0% 9 way one-shot learning accuracy 


 ------------- 

Time for 400 iterations: 1.2416121204694113 mins
Train Loss: 0.5158249139785767
Evaluating model on 100 random 9 way one-shot learning tasks ... 

Got an average of 89.0% 9 way one-shot learning accuracy 


 ------------- 

Time for 500 iterations: 1.909624163309733 mins

##Thus we got an average accuracy of 70%